In [55]:
import random
substitutionMatchReward = 3
substitutionNotMatchPenalty = -3
horizontalMovementPenalty = -2
verticalMovementPenalty = -2


def getWords(s1,s2):
	return s1.split(), s2.split()


def initialize(matrix,traceMatrix,fileList1,fileList2):
	## first row and column will be 0 automatically
	##global matrix,traceMatrix
	for i in range(len(matrix)):
		for j in range(len(matrix[i])):
			if j == 0 or i ==0:
				matrix[i][j] = 0
	##Dikkat 1den başlıyolar
	for i in range(1,len(matrix)):
		for j in range(1,len(matrix[i])):
			##print("asdasd: ", fileList1[i-1])
			matrix[i][j], traceMatrix[i][j] = calculateIndexValue(matrix[i-1][j-1],matrix[i-1][j],matrix[i][j-1],fileList1[i-1],fileList2[j-1])
	return matrix,traceMatrix	
	##return matrix

"""						  i(Baba)
				| upleft | up  | ...
	    j(Anne)	| left   | ?   | ...
"""
def calculateIndexValue(upLeft,up,left,i,j):
	
	if upLeft == None:
		upLeft = 0
	if up == None:
		up = 0
	if left == None:
		left = 0

	left += horizontalMovementPenalty
	up += verticalMovementPenalty
##	print("i: ",i)
##	print("j: ",j)
	if i == j:
		upLeft += substitutionMatchReward
	else:
		upLeft += substitutionNotMatchPenalty
##	print("up,u,l: ", upLeft,up,left)

	if left < 0:
		left = 0
	if up < 0:
		up = 0
	if upLeft < 0:
		upLeft = 0
	maximum = max(upLeft,up,left)	
	if maximum == upLeft:
		return upLeft,0
	elif maximum == up:
		return up,1
	elif maximum == left:
		return left,2

def findMax(matrix):
	maximum = 0
	iIndex = -1
	jIndex = -1
	for i in range(0,len(matrix)):
		counter = 0
		for j in range(0,len(matrix[i])):
			if matrix[i][j] == None:
				continue
			if matrix[i][j] > maximum:
				maximum = matrix[i][j]
				jIndex = j
				iIndex = i
	return maximum, iIndex, jIndex				


def printf(matrix,s1,s2):
	print()
	print('\t',end='')
	for i in s2:
		print('\t', i, end='')
	print()
	for i in range(0,len(matrix)-1):
		print(s1[i], end='')
		for j in range(0,len(matrix)-1):
			print("\t",matrix[i][j],end=' ')	
		print("\n")		
	print()
	print()
def printf(matrix):
	print()
	for i in range(0,len(matrix)-1):
		for j in range(0,len(matrix)-1):
			print("\t",matrix[i][j],end=' ')	
		print("\n")		


def printSimilarity(matrix,traceMatrix,fileList1,fileList2):
	similarText=''
	similarText2=''
	maximum,i,j= findMax(matrix)

	if maximum == 0:
		return -1,-1,-1
	while 1:
		##print("i: ",i, "j: ", j)
		if matrix[i][j] == 0:
			break
		similarText = fileList1[i-1]
		similarText2 = fileList2[j-1]
		if traceMatrix[i][j] == 0:
			i = i - 1
			j = j - 1
		elif traceMatrix[i][j] == 1:
			i = i - 1
		else: ##2
			j = j - 1	
	return similarText,i+1,j+1

def updateLog(duplicateLog,newlog,file1,file2,line1,line2):
	##global duplicateLog
	if len(duplicateLog) == 0:
		duplicateLog.append(newlog.strip()+':  '+ '['+str(file1)+'.txt' +' ('+str(line1)+') '+ '- '+str(file2)+'.txt'+' ('+str(line2)+')'+'] -'+ str(file1*file2))
		return
	for i in duplicateLog:
		if str(file1*file2) in i:
			##print("here", file1*file2)
			continue
		else:
			##print("added: ",file1*file2 )
			duplicateLog.append(newlog.strip()+':  '+ '['+str(file1)+'.txt' +' ('+str(line1)+') '+ '- '+str(file2)+'.txt'+' ('+str(line2)+')'+'] -'+ str(file1*file2))
	return duplicateLog
def compareTwoFile(fileN1,fileN2):
	##global matrix,traceMatrix,fileList1,fileList2
	duplicateLog = []
	file1 = open(fileN1, 'r',encoding="utf8")
	fileList1 = file1.readlines()
	file1Len = len(fileList1)+1
	

	file2 = open(fileN2, 'r',encoding="utf8")
	fileList2 = file2.readlines()
	file2Len = len(fileList2)+1

	print(file1Len,"-",file2Len)
	
	matrix = [[None for x in range(file2Len)] for y in range(file1Len)]
	traceMatrix = [[None for x in range(file2Len)] for y in range(file1Len)]
	
	matrix,traceMatrix = initialize(matrix,traceMatrix,fileList1,fileList2)

	maximum,ias,jas= findMax(matrix)
	if maximum > 0:
		similarText,ix,iy = printSimilarity(matrix,traceMatrix,fileList1,fileList2)
		print("Duplicate Found!! in ", fileN1,'(',ix,')'," and ",fileN2,'(',iy,'):')
		updateLog(duplicateLog,similarText,int(fileN1.replace(".txt","")),int(fileN2.replace(".txt","")),ix,iy)
	file2.close()
	file1.close()
	return duplicateLog,matrix,traceMatrix

def compareAllFiles():
	global matrix,traceMatrix,fileList1,fileList2
	for i in range(0,102):
		fileName1 = str(i) + '.txt'
		file1 = open(fileName1, 'r',encoding="utf8")
		fileList1 = file1.readlines()
		file1Len = len(fileList1)+1

		for j in range(0,102):
			if i == j:
				continue
			fileName2 = str(j) + '.txt'
			file2 = open(fileName2, 'r',encoding="utf8")
			fileList2 = file2.readlines()
			file2Len = len(fileList2)+1

			matrix = [[None for x in range(file2Len)] for y in range(file1Len)]
			traceMatrix = [[None for x in range(file2Len)] for y in range(file1Len)]
			matrix,traceMatrix=initialize(matrix,traceMatrix,fileList1,fileList2)
			maximum,ias,jas= findMax(matrix)
			##print(ias ," - ", jas, " -> ", maximum)
			###print(maximum)
			if maximum > 0:
				similarText,ix,iy = printSimilarity(matrix,traceMatrix,fileList1,fileList2)
				print("Duplicate Found!! in ", fileName1,'(',ix,')'," and ",fileName2,'(',iy,'):')
				##duplicateLog = updateLog(duplicateLog,similarText,int(fileName1.replace(".txt","")),int(fileName2.replace(".txt","")),ix,iy)
			file2.close()
		file1.close()


def traceBackMatrix(matrix,traceMatrix):
	maximum,i,j= findMax(matrix)
	im = i
	jm = j
	##print("max: ", maximum)
	while 1:
		if i == 1 or j == 1:
			break
		if matrix[i][j] !=0:	
			matrix[i][j] = None	
		if traceMatrix[i][j] == 0: ## going diagonal. means both have that char
			i-=1
			j-=1
		elif traceMatrix[i][j] == 1: ## going top. means only one has that char
			i-=1
		elif traceMatrix[i][j] == 2: ## going left. means only one has that char
			j-=1
	return matrix,im,jm


def printSmithWaterman(matrix,si,sj,list1,list2):
	final = ""
	write = True
	##if matrix[si][sj] == None:
	##	final+=list2[sj-1]
	while matrix[si][sj] is None:
		##üstü none
		if matrix[si-1][sj] == None:
			final+='_'
			write = False
			si = si-1
		##sağı none
		elif matrix[si][sj-1] == None:
			final+='_'
			write = False
			sj = sj-1
		##çaprazı none
		elif matrix[si-1][sj-1] == None:
			if list1[si-1] == list2[sj-1]:
				final+=list2[sj-1]
			si = si-1
			write = True
			sj = sj-1
		else:
			if matrix[si][sj] == None and write == True:
				final+=list2[sj-1]
			break	
	return final[::-1]



def smithWaterman(text1,text2,isPrint):
	matrix = [[]]
	traceMatrix = [[]]
	list1 = []
	list2 = []
	for i in text1:
		list2.append(i)
	for i in text2:
		list1.append(i)

	matrix = [[None for x in range(len(list2)+1)] for y in range(len(list1)+1)]
	traceMatrix = [[None for x in range(len(list2)+1)] for y in range(len(list1)+1)]

	matrix, traceMatrix = initialize(matrix,traceMatrix,list1,list2)
	if isPrint == True:
		printf(matrix)
	##printf(matrix,list1,list2)
	matrix,i,j = traceBackMatrix(matrix,traceMatrix)
	result = printSmithWaterman(matrix,i,j,list1,list2)

    ##printf(traceMatrix,list1,list2)
	##print()
	print(list1)
	print(list2)
	print(result)


In [56]:
## iki dosya içinde arama
deuplicate,matrix,traceMatrix = compareTwoFile("102.txt","103.txt")
print("---------------------------\n")
printf(matrix)
for i in deuplicate:
	print(i)	


5 - 5
Duplicate Found!! in  102.txt ( 1 )  and  103.txt ( 3 ):
---------------------------


	 0 	 0 	 0 	 0 

	 0 	 0 	 0 	 3 

	 0 	 0 	 0 	 1 

	 0 	 0 	 0 	 0 

A T G H H C T:  [102.txt (1) - 103.txt (3)] -10506


In [57]:
print("example: ")
smithWaterman("ATGCTCTTT","AGCTACTT",isPrint = True)
print("---------------------------\n")
print("example: ")
smithWaterman("TGTTACGG","GGTTGACTA",isPrint = True)
print("---------------------------\n")
print("example: ")
smithWaterman("aaaacccccggggtta","ttcccgggaaccaacc",isPrint = False)	
print("-----------	----------------\n")
print("example: ")
smithWaterman("INSERTION","EXECUTIONS",isPrint = False)

example: 

	 0 	 0 	 0 	 0 	 0 	 0 	 0 	 0 

	 0 	 3 	 1 	 0 	 0 	 0 	 0 	 0 

	 0 	 1 	 0 	 4 	 2 	 0 	 0 	 0 

	 0 	 0 	 0 	 2 	 7 	 5 	 3 	 1 

	 0 	 0 	 3 	 1 	 5 	 10 	 8 	 6 

	 0 	 3 	 1 	 0 	 3 	 8 	 7 	 5 

	 0 	 1 	 0 	 0 	 3 	 6 	 11 	 9 

	 0 	 0 	 4 	 2 	 1 	 6 	 9 	 14 

['A', 'G', 'C', 'T', 'A', 'C', 'T', 'T']
['A', 'T', 'G', 'C', 'T', 'C', 'T', 'T', 'T']
GCT_CTT
---------------------------

example: 

	 0 	 0 	 0 	 0 	 0 	 0 	 0 	 0 	 0 

	 0 	 0 	 3 	 1 	 0 	 0 	 0 	 3 	 3 

	 0 	 0 	 3 	 1 	 0 	 0 	 0 	 3 	 6 

	 0 	 3 	 1 	 6 	 4 	 2 	 0 	 1 	 4 

	 0 	 3 	 1 	 4 	 9 	 7 	 5 	 3 	 2 

	 0 	 1 	 6 	 4 	 7 	 6 	 4 	 8 	 6 

	 0 	 0 	 4 	 3 	 5 	 10 	 8 	 6 	 5 

	 0 	 0 	 2 	 1 	 3 	 8 	 13 	 11 	 9 

	 0 	 3 	 1 	 5 	 4 	 6 	 11 	 10 	 8 

['G', 'G', 'T', 'T', 'G', 'A', 'C', 'T', 'A']
['T', 'G', 'T', 'T', 'A', 'C', 'G', 'G']
GTT_AC
---------------------------

example: 
['t', 't', 'c', 'c', 'c', 'g', 'g', 'g', 'a', 'a', 'c', 'c', 'a', 'a', 'c', 'c']
['a', 'a', 'a', 'a

In [58]:

print("This will search all files and find the common lines!!")
compareAllFiles()

This will search all files and find the common lines!!
Duplicate Found!! in  18.txt ( 45 )  and  28.txt ( 106 ):
Duplicate Found!! in  18.txt ( 45 )  and  79.txt ( 132 ):
Duplicate Found!! in  28.txt ( 106 )  and  18.txt ( 45 ):
Duplicate Found!! in  28.txt ( 106 )  and  79.txt ( 132 ):
Duplicate Found!! in  79.txt ( 132 )  and  18.txt ( 45 ):
Duplicate Found!! in  79.txt ( 132 )  and  28.txt ( 106 ):
